EfficientNet

In [ ]:
print('start')

In [ ]:
!pip install -e /kaggle/input/efficientnet-keras-dataset/efficientnet_kaggle

In [ ]:
#!pip install keras

In [ ]:
import shutil
from collections import defaultdict
import json
from pathlib import Path
import os
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from numpy import asarray
import pandas as pd
import matplotlib.pyplot as plt
# Import from keras_preprocessing not from keras.preprocessingfrom keras_preprocessing.image import ImageDataGenerator
#from tensorflow.keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.regularizers import l1_l2
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D,MaxPooling2D, Dropout,Flatten,Dense,Activation, BatchNormalization
from tensorflow.keras.optimizers import SGD
import tensorflow as tf
from tensorflow import keras
tf.keras.backend.clear_session()
from tensorflow.keras.callbacks import CSVLogger
import time

In [ ]:
from tensorflow.keras import layers, models, Model

In [ ]:
import sys
sys.path.append('/kaggle/input/efficientnet-keras-dataset/efficientnet_kaggle')

In [ ]:
import efficientnet.tfkeras as efn

In [ ]:
#model = efn.EfficientNetB2(weights='imagenet')

In [ ]:
keras.__version__

In [ ]:
tf.__version__

**Separate benchmark testset**

In [ ]:
traindir = '/kaggle/working/train/'
testdir = '/kaggle/working/test/'
tbdir = '/kaggle/working/tb/'
trainset = '/kaggle/working/train.csv'
testset = '/kaggle/working/test.csv'
tbset = '/kaggle/working/tb.csv'

In [ ]:
try:
    shutil.copytree(os.path.join('/kaggle/input/petfinder-pawpularity-score/train/'), os.path.join('/kaggle/working/train/'))
    shutil.copytree(os.path.join('/kaggle/input/petfinder-pawpularity-score/test/'), os.path.join('/kaggle/working/test/'))
    shutil.copy(os.path.join('/kaggle/input/petfinder-pawpularity-score/train.csv'), os.path.join('/kaggle/working/train.csv'))
    shutil.copy(os.path.join('/kaggle/input/petfinder-pawpularity-score/test.csv'), os.path.join('/kaggle/working/test.csv'))
except:
    print('Files exist')

In [ ]:
def select_test_benchmark(src, nb, dest, tbfile):
    """
    src: sourcedirectory to be used, i.e. 'train'
    nb: number of images, i.e. 1000
    dest: name of the destination directory, i.e. 'tb'
    tbfile: name of csv file that has names
    """
    np.random.seed(345)

    os.makedirs(os.path.join(dest), exist_ok=True)
    print('Number of files already in destination:', len(os.listdir(dest)))
    
    if len(os.listdir(dest)) == 0:
        selected_files = np.random.choice(os.listdir(src), nb, replace=False)
            
        for file_name in selected_files:
            if os.path.isfile(os.path.join(src, file_name)):
                try:
                    shutil.move(os.path.join(src, file_name), os.path.join(dest, file_name))
                except:
                    print("Error for file", file_name)
        print('Number of files copied:', len(os.listdir(dest)))
                
        df_tb = pd.DataFrame({'tb': selected_files})
        df_tb.to_csv(tbfile)
    else:
        print('No copy-paste is executed')

In [ ]:
select_test_benchmark(traindir, 500, tbdir, tbset)

In [ ]:
def count_images(sourcedir, catnames):
    """catnames: for example ['classic', 'country']"""
    for catname in catnames:
        src = os.path.join(sourcedir, catname)
        print(catname, len(os.listdir(src)))

In [ ]:
count_images('/kaggle/working/', ['train', 'test', 'tb'])

In [ ]:
train_df = pd.read_csv(trainset, dtype=str)
test_df = pd.read_csv(testset, dtype=str)
tb_names = pd.read_csv(tbset, dtype=str)

In [ ]:
def append_ext(fn):
    return fn+".jpg"

In [ ]:
train_df["Id"]=train_df["Id"].apply(append_ext)
test_df["Id"]=test_df["Id"].apply(append_ext)

**Split off the tbset**

In [ ]:
tb_df = train_df[train_df['Id'].isin(list(tb_names['tb']))] # tb cases separated
train_df = train_df[~train_df['Id'].isin(tb_names['tb'])] # tb cases removed from train_df

**Pawpularity index**

In [ ]:
train_df['Pawpularity'] = pd.to_numeric(train_df['Pawpularity'])
tb_df['Pawpularity'] = pd.to_numeric(tb_df['Pawpularity'])

**Check if names tb_df correspond with train_df**

In [ ]:
def name_to_number(inputlist, character):
    """
    counts the position of a character in each string and finally sums all positions to create a unique number.
    inputlist: for ex l1 = ['hellobrother', 'bbf', 'erg', 'ttzee']
    character: for ex 'e'
    statistic: for ex sum
    """
    tl = []
    for x in range(0, len(inputlist)):
        tl.append(inputlist[x].find(character))
    print(sum(tl))

In [ ]:
name_to_number(os.listdir('tb/'), 'z')

In [ ]:
name_to_number(list(tb_df['Id']), 'z')

**Keras**

In [ ]:
isize = 300 # B0: 224
BATCH_SIZE = 32

In [ ]:
datagen=ImageDataGenerator(rescale=1./255., horizontal_flip=False, width_shift_range=False, brightness_range=None, validation_split=0.20)
train_generator=datagen.flow_from_dataframe(dataframe=train_df,
    directory=traindir,
    x_col="Id",
    y_col="Pawpularity",
    subset="training",
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=True,
    class_mode="raw",
    target_size=(isize,isize))

valid_generator=datagen.flow_from_dataframe(rescale=1./255., dataframe=train_df,
    directory=traindir,
    x_col="Id",
    y_col="Pawpularity",
    subset="validation",
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=True,
    class_mode="raw",
    target_size=(isize,isize))

test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=testdir,
    x_col="Id",
    y_col=None,
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=False,
    class_mode=None,
    target_size=(isize,isize))

tb_generator=test_datagen.flow_from_dataframe(
    dataframe=tb_df,
    directory=tbdir,
    x_col="Id",
    y_col="Pawpularity",
    batch_size=32,
    seed=42,
    shuffle=False,
    class_mode="raw",
    target_size=(isize,isize))

In [ ]:
#tf.config.list_physical_devices('CPU') # tf.test.is_gpu_available() will be deprecated

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
#isize=224

In [ ]:
# model = efn.EfficientNetB0(weights='imagenet')

In [ ]:
inputs = layers.Input(shape=(isize, isize, 3))
model = efn.EfficientNetB3(include_top=False, input_tensor=inputs)
model.trainable = False
x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
x = layers.BatchNormalization()(x)
top_dropout_rate = 0.4
x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
outputs = layers.Dense(1, name="pred")(x)
model = keras.Model(inputs, outputs, name="EfficientNet")

In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=[tf.keras.metrics.RootMeanSquaredError()])

#model.summary()

In [ ]:
import time

In [ ]:
t1 = time.time()
# Early stopping to prevent overtraining and to ensure decreasing validation loss
early_stop = EarlyStopping(monitor='val_loss',
                           patience=10,
                           restore_best_weights=True,
                           mode='min')

csv_logger = CSVLogger('training.log', separator=',', append=False)

# Actual fitting of the model

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

history = model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    callbacks=[early_stop, csv_logger],
                    epochs=28, workers=4)
t2 = time.time()
print('time taken (min):', (t2-t1)/60)

**Plot learning curve**

In [ ]:
def savefigure(name):
    """name: give the name of the figure, for ex. Petimages-L1-2021-5-15-15h15"""
    # summarize history for accuracy
    plt.plot(history.history['root_mean_squared_error'])
    plt.plot(history.history['val_root_mean_squared_error'])
    plt.title('model root_mean_squared_error: '+name)
    plt.ylabel('root_mean_squared_error')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper right')
    #plt.savefig(name+'.jpg')
    plt.show()

In [ ]:
savefigure('chk')

In [ ]:
rmse = np.sqrt(model.evaluate(tb_generator, verbose=0))
print("test benchmark (tb) RMSE: {:.5f}".format(rmse[0]))

In [ ]:
from sklearn.metrics import r2_score

r2 = r2_score(true_pawpularity, predicted_pawpularity)
print("Test R^2 Score: {:.5f}".format(r2))

In [ ]:
model_predict = model.predict(test_generator)

In [ ]:
model_predict

In [ ]:
subm = pd.DataFrame(model_predict, index=test_generator.filenames).reset_index()
subm.columns = ['Id', 'Pawpularity']

In [ ]:
subm.head()

In [ ]:
subm['Id'] = subm['Id'].apply(lambda x: x[:-4])

In [ ]:
subm = subm[['Id', 'Pawpularity']]
subm.head()

In [ ]:
subm.to_csv('/kaggle/working/submission.csv', index=False)

In [ ]:
ls '/kaggle/working/submission.csv'

In [ ]:
ls /kaggle/working/